In [1]:
import pandas as pd
import os
import sys

In [2]:
data_path = "/Users/kahaan/Desktop/multi-armed-bandits/data/"

notebook_dir = os.path.dirname(os.path.realpath('__file__'))
base_directory = os.path.normpath(os.path.join(notebook_dir, '..'))
if base_directory not in sys.path:
    sys.path.append(base_directory)

data_directory = os.path.join(base_directory, 'data')
if not os.path.isdir(data_directory):
    raise Exception(f"The directory {data_directory} does not exist.")

In [4]:
train_path = data_path + "training_ratings.txt"
test_path = data_path + "testing_ratings.txt"
title_path = data_path + "movie_titles.txt"
genre_path = data_path + "movie_genres.csv"

# Read in movie titles line by line
titles = []
with open(title_path, 'r', encoding='latin1') as file:
    for line in file:
        # Maximum of 2 splits per line, avoiding commas in movie name
        parts = line.strip().split(',', 2)
        # Verify succesful split
        if len(parts) == 3:
            titles.append(parts)
        else:
            print(f"Skipped malformed line: {line}")

# Create DataFrame for raw data
df_titles = pd.DataFrame(titles, columns=['MovieID', 'Year', 'Title'])
df_train = pd.read_csv(train_path, header=None, names=["MovieID", "UserID", "Rating"])
df_test = pd.read_csv(test_path, header=None, names=["MovieID", "UserID", "Rating"])

# Combine similar formatted data, and set correct data types for later merges
df_combined = pd.concat([df_train, df_test], ignore_index=True)
df_genres = pd.read_csv(genre_path, names=["MovieID", "Genres"]).drop(index=0)
df_combined['MovieID'] = df_combined['MovieID'].astype(int)
df_genres['MovieID'] = df_genres['MovieID'].astype(int)
df_titles['MovieID'] = df_titles['MovieID'].astype(int)

# Merge datasets
df_merge1 = pd.merge(df_combined, df_genres, on='MovieID', how='inner')
df_subset = pd.merge(df_merge1, df_titles, on='MovieID', how='inner')

# Check entries per user
entries_per_user = df_subset.groupby('UserID').size()
smallest_entries_count = entries_per_user.min()
print(f"The smallest amount of entries for any UserID is: {smallest_entries_count}")

df_subset

The smallest amount of entries for any UserID is: 39


,MovieID,UserID,Rating,Genres,Year,Title
0,8,1744889,1.0,Documentary|Comedy|Drama|Fantasy|Mystery|Sci-Fi,2004,What the #$*! Do We Know!?
1,8,1395430,2.0,Documentary|Comedy|Drama|Fantasy|Mystery|Sci-Fi,2004,What the #$*! Do We Know!?
2,8,1205593,4.0,Documentary|Comedy|Drama|Fantasy|Mystery|Sci-Fi,2004,What the #$*! Do We Know!?
3,8,1488844,4.0,Documentary|Comedy|Drama|Fantasy|Mystery|Sci-Fi,2004,What the #$*! Do We Know!?
4,8,1447354,1.0,Documentary|Comedy|Drama|Fantasy|Mystery|Sci-Fi,2004,What the #$*! Do We Know!?
...,...,...,...,...,...,...
2891053,17742,1898310,2.0,Biography|Drama|History|Romance,1995,Catherine the Great
2891054,17742,716096,4.0,Biography|Drama|History|Romance,1995,Catherine the Great
2891055,17742,38115,3.0,Biography|Drama|History|Romance,1995,Catherine the Great
2891056,17742,2646347,5.0,Biography|Drama|History|Romance,1995,Catherine the Great


In [3]:
# Convert large .txt file into structured DataFrame
def transform_file_to_dataframe(file_path):
    transformed_data = []
    with open(file_path, 'r') as file:
        current_movie_id = None  
        for line in file:
            line = line.strip()
            if line.endswith(':'):  
                # If current line is movie ID, strip colon to capture ID
                current_movie_id = line[:-1]  
            else:
                # Otherwise, split by commas and prepend most recent ID
                user_id, rating, date = line.split(',')
                transformed_data.append([current_movie_id, user_id, rating, date])
                
    columns = ['MovieID', 'UserID', 'Rating', 'Date']
    dataframe = pd.DataFrame(transformed_data, columns=columns)
    return dataframe

# Full datasets available at https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data/data
df_1 = transform_file_to_dataframe(data_path + "combined_data_1.txt")
df_2 = transform_file_to_dataframe(data_path + "combined_data_2.txt")
df_3 = transform_file_to_dataframe(data_path + "combined_data_3.txt")
df_4 = transform_file_to_dataframe(data_path + "combined_data_4.txt")

In [4]:
# Merge datasets
df_12 = pd.concat([df_1, df_2])
df_34 = pd.concat([df_3, df_4])
df_1234 = pd.concat([df_12, df_34])
df_1234["MovieID"] = df_1234["MovieID"].astype(int)
df_1234_genres = pd.merge(df_1234, df_genres, on="MovieID", how="left")
df_titles["MovieID"] = df_titles["MovieID"].astype(int)
df_final = pd.merge(df_1234_genres, df_titles, on="MovieID", how="left")

# Check entries per user
entries_per_user = df_final.groupby('UserID').size()
smallest_entries_count = entries_per_user.min()
print(f"The smallest amount of entries for any UserID is: {smallest_entries_count}")

df_final

The smallest amount of entries for any UserID is: 1


,MovieID,UserID,Rating,Date,Genres,Year,Title
0,1,1488844,3,2005-09-06,Documentary|Animation|Family,2003,Dinosaur Planet
1,1,822109,5,2005-05-13,Documentary|Animation|Family,2003,Dinosaur Planet
2,1,885013,4,2005-10-19,Documentary|Animation|Family,2003,Dinosaur Planet
3,1,30878,4,2005-12-26,Documentary|Animation|Family,2003,Dinosaur Planet
4,1,823519,3,2004-05-03,Documentary|Animation|Family,2003,Dinosaur Planet
...,...,...,...,...,...,...,...
100480502,17770,1790158,4,2005-11-01,Action|Adventure|Sci-Fi|Thriller,2003,Alien Hunter
100480503,17770,1608708,3,2005-07-19,Action|Adventure|Sci-Fi|Thriller,2003,Alien Hunter
100480504,17770,234275,1,2004-08-07,Action|Adventure|Sci-Fi|Thriller,2003,Alien Hunter
100480505,17770,255278,4,2004-05-28,Action|Adventure|Sci-Fi|Thriller,2003,Alien Hunter
